In [5]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt

In [45]:
df = pd.read_json('SDU_Housing.json')

df = json_normalize(df['messages'])
channel_name = df['from'].dropna().iloc[0]
df = df[['date', 'text_entities']]

Аренда квартира Алматы


In [23]:
df.head()

,date,text_entities
0,2021-01-23T22:33:10,[]
1,2021-09-06T14:31:17,[]
2,2021-09-06T14:31:17,[]
3,2021-09-06T14:31:17,[]
4,2021-09-06T14:31:17,"[{'type': 'plain', 'text': 'Сдам двухкомнатную..."


In [27]:
df = df[df['text_entities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

def extract_message(text_entities):
    try:
        # Concatenate the 'text' fields from each dictionary in the list
        return ''.join([entity['text'] for entity in text_entities if 'text' in entity])
    except:
        return None

# Apply the function to create a new 'message' column
df['message'] = df['text_entities'].apply(extract_message)

# Drop the 'text_entities' column as it is no longer needed
df_cleaned = df.drop(columns=['text_entities'])

In [65]:
df_cleaned = df_cleaned[df_cleaned['date'] >= '2024-10-10']
df_cleaned = df_cleaned.head(10)

In [66]:
from gemini1 import gemini

message = df_cleaned['message'].iloc[9] + "   " + channel_name
result = gemini(message)
print(result)

city: Алматы
district: null
cost: 67000
gender: female
people_needed: 3
long_term: null
phone_number: 87073159210
utilities: null
deposit: 16000


In [67]:
import json
from gemini1 import gemini
from parse import parse_string
import re

# Initialize lists to store the extracted values for each column
cities = []
districts = []
addresses = []
costs = []
genders = []
people_needed = []
long_terms = []
phone_numbers = []
utilities = []
deposits = []

# Iterate over the first 10 rows of df_cleaned
for i in range(10):
    message = df_cleaned['message'].iloc[i] + " " + channel_name
    result = gemini(message)  # Process the message with the gemini function
    
    if result:
        structured_data = parse_string(result)  # Parse the structured string

        # Append the extracted values to the respective lists
        cities.append(structured_data['city'])
        districts.append(structured_data['district'])
        addresses.append(None)  # Assuming no address in the result string for now
        costs.append(structured_data['cost'])
        genders.append(structured_data['gender'])
        people_needed.append(structured_data['people_needed'])
        long_terms.append(structured_data['long_term'])
        phone_numbers.append(structured_data['phone_number'])
        utilities.append(structured_data['utilities'])
        deposits.append(structured_data['deposit'])
    else:
        # Append None for each field if the result is empty or invalid
        cities.append(None)
        districts.append(None)
        addresses.append(None)
        costs.append(None)
        genders.append(None)
        people_needed.append(None)
        long_terms.append(None)
        phone_numbers.append(None)
        utilities.append(None)
        deposits.append(None)

# Add the new columns to df_cleaned
df_cleaned['city'] = cities
df_cleaned['district'] = districts
df_cleaned['address'] = addresses
df_cleaned['cost'] = costs
df_cleaned['gender'] = genders
df_cleaned['people_needed'] = people_needed
df_cleaned['long_term'] = long_terms
df_cleaned['phone_number'] = phone_numbers
df_cleaned['utilities'] = utilities
df_cleaned['deposit'] = deposits

# Display the updated DataFrame
df_cleaned


                      date                                            message  \
15289  2024-10-10T00:31:32  Заселиться нужно до 15 октября.\nНужна квартир...   
15290  2024-10-10T01:35:08  🎁 Осы топтың барлық жаңа пайдаланушылары үшін ...   
15291  2024-10-10T09:39:26  СРОЧНО\nПодселениеге 2 қыз іздеймін.\nОплата а...   
15292  2024-10-10T09:39:39  2х комнаталык кв 1 комнатасына 2   кыз или 2 у...   
15293  2024-10-10T09:46:00  Алматы қаласы наурысбайски раион алма сити жк ...   
15294  2024-10-10T12:10:35  Сдам  комнату платежеспособной и  аккуратной д...   
15297  2024-10-10T13:21:27  Доброго дня 🪐 В двухкомнатной квартире сдается...   
15298  2024-10-10T16:20:58  🚨Срочно🚨\nСдаём  однокомнатную кв в хорошем со...   
15304  2024-10-10T18:56:48  всем привет! ищем 2х девочек на подселение! \n...   
15305  2024-10-10T20:09:40  Кв 3 қыз керек срочно бүгін или ертен көшіп ке...   

         city             district address      cost  gender  people_needed  \
15289  Алматы           Алата

In [68]:
df_cleaned

,date,message,city,district,address,cost,gender,people_needed,long_term,phone_number,utilities,deposit
15289,2024-10-10T00:31:32,Заселиться нужно до 15 октября.\nНужна квартир...,Алматы,Алатауский,None,130000.0,null,2.0,None,87055379728,None,NaN
15290,2024-10-10T01:35:08,🎁 Осы топтың барлық жаңа пайдаланушылары үшін ...,Алматы,null,None,NaN,null,NaN,None,None,None,NaN
15291,2024-10-10T09:39:26,СРОЧНО\nПодселениеге 2 қыз іздеймін.\nОплата а...,Алматы,null,None,60000.0,female,2.0,None,87763133090,true,30000.0
15292,2024-10-10T09:39:39,2х комнаталык кв 1 комнатасына 2 кыз или 2 у...,Алматы,null,None,NaN,female,2.0,None,87071880474,None,NaN
15293,2024-10-10T09:46:00,Алматы қаласы наурысбайски раион алма сити жк ...,Алматы,Наурызбайский район,None,50000.0,female,1.0,None,87007316391,None,NaN
15294,2024-10-10T12:10:35,Сдам комнату платежеспособной и аккуратной д...,Алматы,Медеуский,None,60000.0,female,1.0,true,None,true,NaN
15297,2024-10-10T13:21:27,Доброго дня 🪐 В двухкомнатной квартире сдается...,Алматы,null,None,100000.0,female,1.0,None,8,true,30000.0
15298,2024-10-10T16:20:58,🚨Срочно🚨\nСдаём однокомнатную кв в хорошем со...,Алматы,Райымбек,None,170000.0,null,NaN,None,87475368386,true,100000.0
15304,2024-10-10T18:56:48,всем привет! ищем 2х девочек на подселение! \n...,Алматы,Бостандыкский,None,62500.0,female,2.0,None,87002219895,true,NaN
15305,2024-10-10T20:09:40,Кв 3 қыз керек срочно бүгін или ертен көшіп ке...,Алматы,null,None,67000.0,female,3.0,None,87073159210,None,16000.0
